In [1]:
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
import datetime
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from scipy import misc, ndimage
from PIL import Image
from keras.optimizers import Adam

In [2]:
train_path = 'pothole_detection/dataset/train'
test_path = 'pothole_detection/dataset/test'
valid_path = 'pothole_detection/dataset/validation'

In [3]:
input_shape = (120, 120)

In [4]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size = input_shape, classes=['pothole', 'no_pothole'], batch_size = 15)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size = input_shape, classes=['pothole', 'no_pothole'], batch_size = 8)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path, target_size = input_shape, classes=['pothole', 'no_pothole'], batch_size = 15)

Found 1712 images belonging to 2 classes.
Found 197 images belonging to 2 classes.
Found 58 images belonging to 2 classes.


In [11]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    channel_shift_range = 0.2,
    rotation_range = 0.1,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(120, 120),
        batch_size=15,
        classes=['pothole', 'no_pothole'],
)

test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(120, 120),
        batch_size=8,
        classes=['pothole', 'no_pothole'],
)

valid_generator = valid_datagen.flow_from_directory(
        valid_path,
        target_size=(120, 120),
        batch_size=15,
        classes=['pothole', 'no_pothole'],
)

Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [5]:
model2 = Sequential()

model2.add(Conv2D(64, (3, 3), activation = 'relu'))
model2.add(MaxPooling2D((2, 2)))

model2.add(Flatten())

model2.add(Dense(256, input_shape = (120, 120, 3), activation = 'relu'))
model2.add(Dropout(0.2))

model2.add(Dense(512, input_shape = (120, 120, 3), activation = 'relu'))
model2.add(Dropout(0.2))

model2.add(Dense(1024, input_shape = (120, 120, 3), activation = 'relu'))
model2.add(Dropout(0.8))

model2.add(Dense(512, input_shape = (120, 120, 3), activation = 'relu'))
model2.add(Dropout(0.2))

model2.add(Dense(256, input_shape = (120, 120, 3), activation = 'relu'))
model2.add(Dropout(0.2))

model2.add(Dense(2, activation = 'softmax'))

In [6]:
model2.compile(Adam(lr = 0.0001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [7]:
model2.fit_generator(train_generator, epochs = 5, steps_per_epoch = 115, validation_data = test_generator, validation_steps = 24)

Instructions for updating:
Please use Model.fit, which supports generators.


ValueError: Asked to retrieve element 0, but the Sequence has length 0

In [8]:
valid_images, valid_labels = next(valid_batches)
valid_labels = valid_labels[:,0]

NameError: name 'valid_batches' is not defined

In [47]:
prediction = model2.predict_generator(valid_generator, steps = 1)
prediction

Instructions for updating:
Please use Model.predict, which supports generators.


array([[0.51112145, 0.4888785 ],
       [0.50723386, 0.4927661 ],
       [0.51030594, 0.48969403],
       [0.5068996 , 0.49310046],
       [0.49130848, 0.5086916 ],
       [0.50716823, 0.49283183],
       [0.5071663 , 0.4928336 ],
       [0.50703573, 0.4929643 ],
       [0.50043446, 0.4995655 ],
       [0.5082463 , 0.49175367],
       [0.5035212 , 0.49647883],
       [0.5055176 , 0.4944823 ],
       [0.50225514, 0.49774483],
       [0.5138278 , 0.48617217],
       [0.50320226, 0.49679777]], dtype=float32)